In [1]:
%load_ext sql
%sql mysql+pymysql://root:@fe512_mysql/fe512db
%sql USE fe512db; 
%sql SELECT DATABASE();

In [ ]:
%%sql
SHOW TABLES;

In [ ]:
%%sql
CREATE TABLE transactions(
  `msno` VARCHAR(44), 
  `payment_method_id` INT ,
  `payment_plan_days` INT,
  `plan_list_price` INT ,
  `actual_amount_paid` INT,
  `is_auto_renew` INT,
  `transaction_date` INT,
  `membership_expire_date` INT,
  `is_cancel` INT
);

In [ ]:
%%sql
LOAD DATA INFILE '/home/data/transactions.csv'  INTO TABLE transactions
FIELDS TERMINATED BY ',' 
ENCLOSED BY '"' 
LINES TERMINATED BY '\n'
IGNORE 1 LINES (msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel);            

In [ ]:
%%sql
CREATE TABLE members(
  `msno` VARCHAR(44) PRIMARY KEY, 
  `city` INT ,
  `bday` INT,
  `gender` TEXT ,
  `registered_via` INT,
  `registrition_init_time` INT
);

In [ ]:
%%sql
LOAD DATA INFILE '/home/data/members.csv'  INTO TABLE members
FIELDS TERMINATED BY ',' 
ENCLOSED BY '"' 
LINES TERMINATED BY '\n'
IGNORE 1 LINES (msno,city,bday,gender,registered_via,registrition_init_time);

In [ ]:
%%sql
CREATE TABLE churn(
  `msno` VARCHAR(44) PRIMARY KEY, 
  `is_churn` INT 
);

In [ ]:
%%sql
LOAD DATA INFILE '/home/data/churn.csv'  INTO TABLE churn
FIELDS TERMINATED BY ',' 
ENCLOSED BY '"' 
LINES TERMINATED BY '\n'
IGNORE 1 LINES (msno,is_churn);               

In [ ]:
%%sql
CREATE TABLE userlog(
  `msno` VARCHAR(44), 
  `date` INT ,
  `num_unq` INT,
  `total_secs` DOUBLE 
);

In [ ]:
%%sql
LOAD DATA INFILE '/home/data/behavior.csv'  INTO TABLE userlog
FIELDS TERMINATED BY ',' 
ENCLOSED BY '"' 
LINES TERMINATED BY '\n'
IGNORE 1 LINES (msno,date,num_unq,@total_secs)
set
total_secs=NULLIF(@total_secs,'');

In [ ]:
%%sql
ALTER TABLE userlog ADD PRIMARY KEY (msno,date);

### Data manipulation

In [ ]:
%%sql
DESCRIBE members;

In [ ]:
%%sql
SELECT COUNT(*) 
FROM members 
WHERE gender = '';

In [ ]:
%%sql
SELECT COUNT(*) 
FROM members 
WHERE bday <= 0 or bday >= 100;

#### The total number of records in Members Table is 6769473.

In [ ]:
%%sql
CREATE TABLE memberchurn AS(SELECT churn.msno, is_churn, members.bday, members.gender
FROM churn JOIN members
ON churn.msno=members.msno
WHERE members.bday>0 and members.bday<100 and members.gender!='')
LIMIT 10;

Q1: Is there a relationship between information integrity and churn?

In [ ]:
%%sql
SELECT COUNT(*)
FROM churn
WHERE is_churn=0;

In [ ]:
%%sql
SELECT COUNT(*)
FROM memberschurn
WHERE is_churn=0;

In [ ]:
%%sql
SELECT COUNT(*)
FROM churn
WHERE is_churn=1;

In [ ]:
%%sql
SELECT COUNT(*)
FROM memberschurn
WHERE is_churn=1;

We can calculate that among all churn users, 36.96% users with completed information. Therefore, we may conclude that there is not significant relation between the information integrity and churn probability.

## Userlog

In [ ]:
%%sql
SELECT max(date), min(date) 
FROM userlog;

In [ ]:
%%sql
DELETE FROM userlog 
WHERE msno NOT IN (SELECT msno FROM churn);

Q2: Is there a relationship between user's listening behavior and churn?

In [ ]:
%%sql
CREATE TABLE user_1_log
(SELECT msno, count(date) as days,sum(num_unq) as total_unq, avg(total_secs) as avg_secs
FROM userlog
GROUP BY msno);

In [ ]:
%%sql
ALTER TABLE user_1_log 
ADD avg_min double;

In [ ]:
%%sql
ALTER TABLE user_1_log 
ADD avg_unq double;

In [ ]:
%%sql
UPDATE user_1_log
SET avg_unq = total_unq/days,avg_min = avg_secs/60;

In [ ]:
%%sql
CREATE TABLE userlog_churn(
SELECT churn.msno,days,avg_unq,avg_min,is_churn 
FROM user_1_log join churn on 
    user_1_log.msno = churn.msno 
    ORDER BY avg_min DESC);

We can see that most of users who are not churn spend more time on listening to music in a month. We may assume that there exsit a relationship between users' behavior and churn probability.

## Transaction

In [ ]:
%%sql
DELETE FROM transactions 
WHERE msno NOT IN (select msno FROM churn);

Q3: Is there a relationship between the number of transactions and churn?

In [ ]:
%%sql
SELECT COUNT(*) 
FROM transactions;

#### The original count of record in Transaction is 21547746.

In [ ]:
%%sql
CREATE TABLE transaction_count(
    SELECT msno,COUNT(*) AS trans_count FROM transactions GROUP BY msno);

In [ ]:
%%sql
CREATE TABLE churn_vs_count(
    SELECT churn.msno, is_churn, trans_count 
    FROM churn JOIN transaction_count ON churn.msno = transaction_count.msno 
    ORDER BY trans_count DESC); 

In [ ]:
%%sql
SELECT msno FROM churn_vs_count 
WHERE is_churn=1;

In [ ]:
%%sql
CREATE TABLE churn_percent_trancount(
    SELECT trans_count, sum(is_churn) AS sum_churn, count(msno) AS sum_cust, (sum(is_churn)/count(msno)) AS churn_percent 
    FROM churn_vs_count 
    GROUP BY trans_count); 

In [ ]:
%%sql
SELECT * FROM churn_percent_trancount INTO OUTFILE '/home/data/churn_percent_trancount.csv'
FIELDS 
    terminated by ',' 
    optionally enclosed by '"' 
    escaped by '"'  
    lines terminated by '\r\n';

### get a smaller set of data, get the information of the earliest day user enrolled in.

In [ ]:
%%sql
CREATE TABLE trans_expire(
    SELECT a.msno,a.plan_list_price,a.is_auto_renew,a.membership_expire_date 
    FROM trans10000 a 
    WHERE membership_expire_date = (
    SELECT max(membership_expire_date) FROM trans10000 WHERE msno = a.msno) 
    ORDER BY a.msno
); 

### get a smaller set of data, get the information of the latest day user churn.

In [ ]:
%%sql
CREATE TABLE trans_date(
    SELECT a.msno,a.plan_list_price, a.is_auto_renew,a.transaction_date 
    FROM trans10000 a where transaction_date = (
    SELECT min(transaction_date) FROM trans10000 WHERE msno = a.msno) 
    ORDER BY a.msno
); 

In [ ]:
%%sql
CREATE TABLE user_duration(
    SELECT trans_date.msno,
    trans_date.transaction_date AS start_date, 
    trans_expire.membership_expire_date AS end_date
    FROM trans_expire JOIN trans_date 
    ON trans_expire.msno = trans_date.msno
);